In [1]:
import zipfile
import numpy as np

import glob
import re
import pandas as pd

# read the dataset using the compression zip
all_data = pd.read_csv('all_data_for_analysis.zip',compression='zip')

In [2]:
pd.set_option('display.max_rows', 100)

In [3]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid
0,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27
1,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62
2,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74
3,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33
4,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109774,84162_rev1,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77
109775,84212_rev1,Logs : https://gist.github.com/Emely Wilson/17...,Emely Wilson,2017-10-10 11:17:15.392000000,NaN,CBQE-4333: Refactor eventing code,1,Emely Wilson,1,http://review.couchbase.org/#/c/84212/1,84212,76,9.21
109776,84284_rev1,Cherry Picked from branch watson.,Monique Barker,2017-10-11 18:40:50.760000000,NaN,Test for MB-25737:group by on empty table shou...,1,Dylan Paul,1,http://review.couchbase.org/#/c/84284/1,84284,34,83.32
109777,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75,-50.02


In [26]:
only_merged_df = all_data[all_data['status']==1]

Remove bots comments

In [ ]:
bot_comments = ['Logs','log','Cherry Picked']
bot_comments_pattern = '|'.join(bot_comments)

Drop nulls in comments

In [27]:
only_merged_df['Comment'].isna().sum()

2

In [28]:
only_merged_df = only_merged_df.dropna(subset=['Comment'])

In [29]:
only_merged_df

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid
0,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27
1,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62
2,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74
3,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33
4,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109774,84162_rev1,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77
109775,84212_rev1,Logs : https://gist.github.com/Emely Wilson/17...,Emely Wilson,2017-10-10 11:17:15.392000000,NaN,CBQE-4333: Refactor eventing code,1,Emely Wilson,1,http://review.couchbase.org/#/c/84212/1,84212,76,9.21
109776,84284_rev1,Cherry Picked from branch watson.,Monique Barker,2017-10-11 18:40:50.760000000,NaN,Test for MB-25737:group by on empty table shou...,1,Dylan Paul,1,http://review.couchbase.org/#/c/84284/1,84284,34,83.32
109777,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75,-50.02


Join all comments from PRs that have been merged

In [42]:
all_comments_merged = ''.join(only_merged_df['Comment'])

In [52]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GNR\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [54]:
r = Rake()
r.extract_keywords_from_text(all_comments_merged)